# SLSTR spatial plotting, quality control and data interrogation

    Version: 2.0
    Date:    10/04/2019
    Author:  Ben Loveday and Hayley Evers-King (Plymouth Marine Laboratory)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as free-to-use in the public domain, with no warranty.
    
    This routine has been designed to work with SLSTR L2 NRT data, which is available as 
    tiles. It can be easily adapted for application to L1 data, but care should be taken
    if using it to analyse L2 NTC data, as this is delivered as a half orbit PDU.

This routine shows examples of how to use python netcdf libraries to ingest Level 2 SLSTR data, mask it according to quality control variables, correct for bias, select only for dual view data and compare it against other, coincident geo-physical variables.

In Python, we usually have two sections of code at the top that occur before we enter the main programme. These
sections typically include:
1. importing modules
2. defining functions that are called in the main programme

Firstly, we begin by importing all of the external modules that we will be using in this script; they are annotated with brief explanations of what they do.

In [1]:
%matplotlib inline

# import tools that allow us access tot system functions, e.g. get working directory, check path.
import os
import shutil

# import high level python system functions
import sys
import warnings

# import tools that let us manipulate dates and times
import datetime

# import tools that let us manipulate arrays (makes Python more like Matlab for matrix operations)
import numpy as np

# import tools that facilitate string pattern matching
import fnmatch

# import tools that let us create log files to write to
import logging

# import tools for netCFD4 manipulation
import xarray as xr

# import tools for plotting, making subplots, and utilising map projections
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# turn off warnings. Bad practice, but we don't want to see them here....
warnings.filterwarnings("ignore")

We will quickly get some map backgrounds for our later plots...

In [2]:
land_resolution = '50m'
land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,
                                        edgecolor='k',
                                        facecolor=cfeature.COLORS['land'])

The next section is defines two paths for where we read our input data from ("MYPATH"), and where we write our log files to (again, current working directory). To help to find your data, please complete the MYPATH variable below with the output generated by the /Configuration_Testing/Data_Path_Checker.ipynb Jupyter notebook in the Configuration_Testing folder.

In [3]:
# e.g. MYPATH = os.path.join("C:/","Users","me","Desktop")
#MYPATH = "<please insert your path from Data_Path_Checker.ipynb here, removing the quotes and chevrons>"
MYPATH = os.path.join("C:\\","Users","docle","Desktop","SST_short_course")
DEFAULT_ROOT_DIR    = os.path.join(MYPATH,'SLSTR_test_data')
DEFAULT_LOG_PATH    = os.getcwd()

# we will be looking for all files that match this pattern
DEFAULT_FILE_FILTER = '*SLSTR*.nc'

#----------------------------------------------------PRE-AMBLE COMPLETE-----------------------------------------------

After all of that preparation, we are at the main entrance point for our code. We begin by defining our logfile so we can record any errors here if things go wrong, or any info/debug messages we may want along the way if we don't want them printed to screen (or to the console in interactive mode).

In [4]:
#-main-------------------------------------------------------------------------
# preliminary stuff
logfile = os.path.join(DEFAULT_LOG_PATH,"SLSTR_test_plot_"+datetime.datetime.now().strftime('%Y%m%d_%H%M')+".log")
# we define a verbose flag to control how much info we want to see. It can also be useful to define a debug flag
# for even more information.
verbose=False

# this option will stop plotting to screen, and instead plot to file
no_show=True
if no_show:
    plt.ioff()

We have defined a log file above, and here we set up how python will use it. Note that this is the first time we use the 'print' command. Print will output its contents to the screen, and here, this output will appear below the box when we run it.

In [5]:
# set file logger
try:
    if os.path.exists(logfile):
        os.remove(logfile)
    print("logging to: "+logfile)
    logging.basicConfig(filename=logfile,level=logging.DEBUG)
except:
    print("Failed to set logger")

logging to: C:\Users\docle\Desktop\SST_short_course\SLSTR\SLSTR_test_plot_20210413_2031.log


So, lets proceed with loading some SLSTR data. The first thing we need to do is find the relevant netCDF files that contain the data that we want to plot. This next block of code collects the names of all netCDF files in our DEFAULT_ROOT_DIR path. We can make this more specific by adapting the DEFAULT_FILE_FILTER variable from "*.nc".

We begin by setting up an empty "list" variables called nc_files, and append to this list as we proceed through a series of loops, defined by the "for" statements.

In [6]:
# -get the files-------------------------------------------------------------
nc_files=[]
for root, _, filenames in os.walk(DEFAULT_ROOT_DIR):
    for filename in fnmatch.filter(filenames, DEFAULT_FILE_FILTER):
        nc_files.append(os.path.join(root, filename))
        if verbose:
            print('Found: '+filename)
        logging.info('Found: '+os.path.join(root, filename))

Lets check what files we have by looping through the list...

In python you can loop through the values in a list using "for item in list:". We only have 1 file in this cacse, though

In [7]:
for nc_file in nc_files:
    print(nc_file)

C:\Users\docle\Desktop\SST_short_course\SLSTR_test_data\S3A_SL_2_WST____20171202T193449_20171202T193749_20171202T205905_0179_025_127_5579_MAR_O_NR_002.SEN3\20171202193449-MAR-L2P_GHRSST-SSTskin-SLSTRA-20171202205905-v02.0-fv01.0.nc


For now, we will just work with the first file we find, which is held in nc_files[0].

In [8]:
nc_file = nc_files[0]

This next line opens our netCDF file. It does not read any data in yet, just makes the contents accessible. We should remember to close this link, especially if we are opening lots of files.

In [9]:
nc_fid = xr.open_dataset(nc_file)

We start by loading our coordinate variables, using the function that we defined at the top of the script. 

note: python is very accepting of white space, but the next line would flag as a problem in a code-checker like pylint. It is spaced like this to make it easy to read.

In [10]:
LON  = nc_fid.lon
LAT  = nc_fid.lat
TIME = nc_fid.adi_dtime_from_sst

And finally we load our data variables and close the netCDF file link. SST is stored as a three dimensional variable, with dimensions of time, lat and lon. The time dimension has length of one, and in order to plot SST as an image, we need to remove this "singleton" dimension. The numpy method "squeeze" will take care of this for us.

In [11]:
SST_raw = np.squeeze(nc_fid.sea_surface_temperature.data)
SST = np.squeeze(nc_fid.sea_surface_temperature.data)
SST_STD_DEV = np.squeeze(nc_fid.sses_standard_deviation.data)
SST_BIAS = np.squeeze(nc_fid.sses_bias.data)
SST_ALG_TYPE = np.squeeze(nc_fid.sst_algorithm_type.data)
WIND_SPEED = np.squeeze(nc_fid.wind_speed.data)

Now we load the quality and masking variables

In [12]:
L2P_FLAGS     = np.squeeze(nc_fid.l2p_flags.data)
QUALITY_LEVEL = np.squeeze(nc_fid.quality_level.data)
nc_fid.close()

Now, lets set up our figure and make an initial plot of our SST data

In [13]:
fig  = plt.figure(figsize=(6,6), dpi=150)
plt.imshow(SST)
if no_show:
    fig.savefig('Plot1_Initial.png',bbox_inches='tight')

Plot successful, but this is ugly, not very helpful as it is not on any geographical map, incorrect as we have not taken abias into account, and has not been quality controlled! Lets take steps to improve this by:
1. reprojecting the data onto a map
2. make a contour plot against our LON and LAT data (we use contourf here as it is faster, but pcolor is more appropriate)
3. apply a more sensible colour bar for SST data
4. adding a colour bar

Then:
1. Masking our data for specific features and qulity values
2. Correcting our the SST for bias and considering the standard deviation
3. Considering dual SST only.

Lastly:
1. Checking associated variables; e.g. wind speed

So, lets perform the steps in the first list. The following code block does exactly that; reprojects (using the basemap toolkit), makes a filled contour plot (using contourf), applies a colour scale (cmap) and adds a colour bar (plt.colorbar).

In [14]:

fig  = plt.figure(figsize=(10,10), dpi=300)

# set projection
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))

# set my vertical plotting order and fontsize
zordcoast=0
fsz=12
SST_plot = SST.copy()
vmin=np.nanmean(SST_plot)-3*np.nanstd(SST_plot)
vmax=np.nanmean(SST_plot)+3*np.nanstd(SST_plot)
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

# plot the data
p1 = plt.pcolormesh(LON,LAT,SST_plot,cmap=plt.cm.jet,vmin=vmin,vmax=vmax)

# add embelishments
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(p1, orientation='horizontal')
cbar.set_label('SST [K]',fontsize=fsz);
plt.title('SLSTR SST [K]', fontsize=fsz);

In [15]:
if no_show:
    fig.savefig('Plot2_SST.png',bbox_inches='tight')

A definite improvement, our data is now accompanied by a scale, with units, and is reprojected so we can relate it to a map.

However, we still have not interrogated our data. 

One of the most important things we need to do with SST data is consider the quality level - so lets start by doing that. 

The next code bloack will display the values of the quality level, stored in the quality level variables in the L2 WST product. Usually, we only consider the product viable where the quality flag is five, but can use quality level 4 in some circumstances.

In [16]:

fig  = plt.figure(figsize=(10*int(np.nanmax(QUALITY_LEVEL))+1,10), dpi=150)
gs  = gridspec.GridSpec(1, int(np.nanmax(QUALITY_LEVEL))+1)
contour_vals = np.arange(np.nanmin(QUALITY_LEVEL)-1,np.nanmax(QUALITY_LEVEL)+1,1)
gs.update(wspace=0.1, hspace=0.1)
# loop through each algorithm
for ii in np.arange(0,int(np.nanmax(QUALITY_LEVEL))+1):
    m = plt.subplot(gs[0,ii], projection=ccrs.PlateCarree(central_longitude=0.0))
    MASKED_QUALITY_LEVEL = QUALITY_LEVEL.astype('float')
    MASKED_QUALITY_LEVEL[MASKED_QUALITY_LEVEL != float(ii)] = np.nan

    # plot the data
    plt.pcolormesh(LON,LAT,np.ma.masked_invalid(MASKED_QUALITY_LEVEL),vmin=0,vmax=5)
    plt.text(33,-17.5,ii,fontweight='bold',fontsize=fsz*2)

    # add embelishments
    m.coastlines(resolution=land_resolution, color='black', linewidth=1)
    m.add_feature(land_poly)
    g1 = m.gridlines(draw_labels = False)
    g1.xlabel_style = {'size': 16, 'color': 'gray'}
    g1.ylabel_style = {'size': 16, 'color': 'gray'}

In [17]:
if no_show:
    fig.savefig('Plot3_Quality.png',bbox_inches='tight')
else:
    plt.show()

So, lets mask out any data that have a quality value of 2 or lower...

In [18]:
SST[QUALITY_LEVEL<=4]=np.nan

Next, the SST field has two associated measurements that we need to consider, the bias, and the standard deviation. So lets plot these...

In [19]:


fig  = plt.figure(figsize=(20,20), dpi=300)
gs  = gridspec.GridSpec(1, 2)
fsz = 20

m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))

# plot the data
SST_plot = SST_BIAS.copy()
vmin=np.nanmean(SST_plot)-3*np.nanstd(SST_plot)
vmax=np.nanmean(SST_plot)+3*np.nanstd(SST_plot)
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

p1 = plt.pcolormesh(LON,LAT,SST_plot,cmap=plt.cm.jet,vmin=vmin,vmax=vmax)
# add embelishments
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(p1, orientation='horizontal', pad=0.05)
cbar.set_label('SST bias [K]',fontsize=fsz)

m = plt.subplot(gs[0,1], projection=ccrs.PlateCarree(central_longitude=0.0))
# plot the data
SST_plot = SST_STD_DEV.copy()
vmin=np.nanmean(SST_plot)-3*np.nanstd(SST_plot)
vmax=np.nanmean(SST_plot)+3*np.nanstd(SST_plot)
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

p1 = plt.pcolormesh(LON,LAT,SST_plot,cmap=plt.cm.jet,vmin=vmin,vmax=vmax)
# add embelishments
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(p1, orientation='horizontal', pad=0.05)
cbar.set_label('SST standard deviation [K]',fontsize=fsz);

In [20]:
if no_show:
    fig.savefig('Plot4_SSES.png',bbox_inches='tight')

The SST value we are interested in needs to be corrected for the bias, so lets do that:

In [21]:
SST = SST + SST_BIAS

We can see from the bias and standard deviation plots that there are some sharp lines across the image. SLSTR uses 5 algorithms to estimate SST. Some of these algorithms use the nadir view only, while some take advantage of the 'Dual View' capability of the sensor. Dual view takes two images of the surface, one at nadir and one at an oblique angle. This allows it to better characterise the effects of the atmosphere. In practice, the nadir view is wider than the dual view, which results in a stripe along the middle of the swath. We can check which algorithms were used to derive the SST estimate by checking the SST_ALG_TYPE variable, as below:

In [22]:


fig  = plt.figure(figsize=(10*int(np.nanmax(SST_ALG_TYPE))+1,10), dpi=150)
gs  = gridspec.GridSpec(1, 6)
gs.update(wspace=0.1, hspace=0.1)
fsz = 12

contour_vals = np.arange(np.nanmin(SST_ALG_TYPE)-1,np.nanmin(SST_ALG_TYPE)+1,1)

# loop through each algorithm
for ii in np.arange(0,int(np.nanmax(SST_ALG_TYPE))+1):
    m = plt.subplot(gs[0,ii], projection=ccrs.PlateCarree(central_longitude=0.0))
    MASKED_ALG_TYPE = SST_ALG_TYPE.astype('float')
    MASKED_ALG_TYPE[MASKED_ALG_TYPE != float(ii)] = np.nan
    # plot the data
    plt.pcolormesh(LON,LAT,np.ma.masked_invalid(MASKED_ALG_TYPE),cmap=plt.cm.jet,vmin=0,vmax=5)
    if ii == 0:
        plt.text(33,-17.5,'No retrieval',fontweight='bold',fontsize=fsz*2)
    if ii == 1:
        plt.text(33,-17.5,'N2',fontweight='bold',fontsize=fsz*2)
    if ii == 2:
        plt.text(33,-17.5,'N3R',fontweight='bold',fontsize=fsz*2)
    if ii == 3:
        plt.text(33,-17.5,'N3',fontweight='bold',fontsize=fsz*2)
    elif ii == 4:
        plt.text(33,-17.5,'D2',fontweight='bold',fontsize=fsz*2)
    elif ii == 5:
        plt.text(33,-17.5,'D3',fontweight='bold',fontsize=fsz*2)

    # add embelishments
    m.coastlines(resolution=land_resolution, color='black', linewidth=1)
    m.add_feature(land_poly)
    g1 = m.gridlines(draw_labels = False)
    g1.xlabel_style = {'size': 16, 'color': 'gray'}
    g1.ylabel_style = {'size': 16, 'color': 'gray'}

In [23]:
if no_show:
    fig.savefig('Plot5_Algorithms.png',bbox_inches='tight')

We should remember that, just because a measurement is Nadir view only, it does not mean that it is bad! Sometimes the nadir view is the best to use. Here, though, lets finally plot our nadir+dual and our dual view data, corrected for bias, and masked for a quality level of 3 or greater. We will overlay the plot with contours from contemporaneous ECMWF wind data, that is included in with SLSTR L2 WAT products, as part of the GHRSST specification.

In [24]:
%%capture


SST_C = SST-273.15

fig  = plt.figure(figsize=(20,20), dpi=150)
gs  = gridspec.GridSpec(3, 1, height_ratios=[20,0.5,1])
gs.update(wspace=0.01, hspace=0.01)

# set my vertical plotting order and fontsize
zordcoast=0
fsz=20

# plot the data
SST_plot = SST_C.copy()
vmin=int(np.nanmean(SST_plot)-3*np.nanstd(SST_plot))-1
vmax=int(np.nanmean(SST_plot)+3*np.nanstd(SST_plot))+1
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))
p1 = plt.contourf(LON,LAT,SST_plot,100,cmap=plt.cm.jet,vmin=vmin,vmax=vmax,zorder=-1)
CS = plt.contour(LON,LAT,WIND_SPEED,10,linewidths=1.0,cmap=plt.get_cmap('Greys'),zorder=0)
plt.clabel(CS, fontsize=10, inline=1)

# add embelishments
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

# add colorbar
axes0 = plt.subplot(gs[2,0])
cbar = plt.colorbar(p1, cax=axes0, orientation='horizontal')
cbar.ax.tick_params(labelsize=fsz) 
cbar.set_label('Bias corrected, quality controlled, whole view SST [$^{o}$C]',fontsize=fsz)

In [25]:
bbox_inches='tight'
if no_show:
    fig.savefig('Plot6_SLSTR_All_SST.png',bbox_inches='tight')

In [26]:
%%capture

SST_C = SST-273.15
SST_C[SST_ALG_TYPE<4] = np.nan

fig  = plt.figure(figsize=(20,20), dpi=150)
gs  = gridspec.GridSpec(3, 1, height_ratios=[20,1,1])
gs.update(wspace=0.01, hspace=0.01)  

# set my vertical plotting order and fontsize
zordcoast=0
fsz=20

# plot the data
SST_plot = SST_C.copy()
vmin=int(np.nanmean(SST_plot)-3*np.nanstd(SST_plot))-1
vmax=int(np.nanmean(SST_plot)+3*np.nanstd(SST_plot))+1
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))
p1 = m.contourf(LON,LAT,SST_plot,100,cmap=plt.cm.jet,vmin=vmin,vmax=vmax,zorder=-1)
CS = m.contour(LON,LAT,WIND_SPEED,10,linewidths=1.0,cmap=plt.get_cmap('Greys'),zorder=0)
plt.clabel(CS, fontsize=14, inline=1)

# add embelishments
m.coastlines(resolution=land_resolution, color='black', zorder=3)
m.add_feature(land_poly, zorder = 2)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

# add colorbar
axes0 = plt.subplot(gs[2,0])
cbar = plt.colorbar(p1, cax=axes0, orientation='horizontal')
cbar.ax.tick_params(labelsize=fsz) 
cbar.set_label('Bias corrected, quality controlled, dual view SST [$^{o}$C]',fontsize=fsz)

In [27]:
bbox_inches='tight'
if no_show:
    fig.savefig('Plot7_SLSTR_Dual_SST_demo.png',bbox_inches='tight')

<br> <a href="./16_OLCI_spectral_AC_L1_L2_comparison.ipynbb"><< 16 - OLCI - atmospheric correction effects: Level-1B to Level-2 spectral comparison</a><span style="float:right;"><a href="../STM/31_SRAL_SLAs.ipynb">31 - Compute corrected sea surface heights & anomalies from Sentinel-3 >></a>  <hr> <p style="text-align:left;">This project is licensed under the <a href="/edit/LICENSE">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eo-lab-usc-open/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:training@eumetsat.int>Contact</a></span></p>